# 06. Baseline Comparison: PyTorch MLP

To evaluate the performance of the **Triglial Reservoir** (which achieved ~71% accuracy on 500 samples), we compare it against a traditional **Multi-Layer Perceptron (MLP)** trained on the exact same data.

## Model Architecture
- **Input**: 784 (28x28 pixels)
- **Hidden**: 100 (ReLU activation) - Same size as the reservoir
- **Output**: 10 (LogSoftmax)
- **Optimizer**: Adam
- **Loss**: CrossEntropyLoss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Load Data (Same Subset)
We use the same subset size (500 training samples) to ensure a fair comparison.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

subset_size = 500
train_subset = Subset(train_data, range(subset_size))
test_subset = Subset(test_data, range(100))

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=100, shuffle=False)

## 2. Define MLP Model

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=100, output_dim=10):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = x.view(-1, 784) # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleMLP()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

## 3. Training Loop

In [ ]:
epochs = 20
losses = []

print(f"Training MLP on {subset_size} samples for {epochs} epochs...")

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    losses.append(epoch_loss / len(train_loader))
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {losses[-1]:.4f}")

plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

## 4. Evaluation

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += target.size(0)

accuracy = correct / total
print(f"Baseline MLP Accuracy: {accuracy * 100:.2f}%")
print(f"Triglial Reservoir Accuracy: ~71.00% (Reference)")